# school_district_pandas_analytics
## deliverable 0: load and clean data

In [236]:
#dependency, load, clean data
import pandas as pd

#filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

#clean student_name data with list, for loop, str.replace
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


## deliverable 1: isolate datapoints + change value w/ loc

In [237]:
#Step 1, 2 and 3: numpy dependency, loc + logical operatiors to pull thomas 9th graders names, change read and math grades to NaN
import numpy as np
stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th"), "reading_score"] = np.nan
stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th"), "math_score"] = np.nan

stu_df.tail(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39160,39160,Katie Weaver,F,11th,Thomas High School,89.0,86.0
39161,39161,April Reyes,F,10th,Thomas High School,70.0,84.0
39162,39162,Derek Weeks,M,12th,Thomas High School,94.0,77.0
39163,39163,John Reese,M,11th,Thomas High School,90.0,75.0
39164,39164,Joseph Anthony,M,9th,Thomas High School,NaN,NaN
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95.0,75.0


## Deliverable 2.1 : district summary analytics

In [238]:
#merge data + district metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

In [239]:
# Step 1. account for THS_niners in total_student_count
#grab THS_niner count and deduct from student_count
only_9_thomas_df = stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th")]
thomas_niner_count = len(only_9_thomas_df["student_name"])
student_count = student_count - thomas_niner_count

In [240]:
# Calculate Dictrict Metrics using the "clean_student_data" + new student counts.
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [241]:
#create district summary df + format
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.1f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.1f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.1f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"38,709","$24,649,428.00",78.9,81.9,74.8,85.7,64.9


## delivery 2.1: school summary

In [242]:
#determine school types
per_school_types = sch_df.set_index(["school_name"])["type"]

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create, format output DF
school_sum_df = pd.DataFrame({
    "School Type" : per_school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})

ssf = school_sum_df.copy()
school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)
school_sum_df["Per Student Budget"] = school_sum_df["Per Student Budget"].map("${:,.2f}".format)
school_sum_df["Ave Math Score"] = school_sum_df["Ave Math Score"].map("{:.1f}".format)
school_sum_df["Ave Read Score"] = school_sum_df["Ave Read Score"].map("{:.1f}".format)
school_sum_df["% Pass Math"] = school_sum_df["% Pass Math"].map("{:.1f}".format)
school_sum_df["% Pass Read"] = school_sum_df["% Pass Read"].map("{:.1f}".format)
school_sum_df["% Overall Pass"] = school_sum_df["% Overall Pass"].map("{:.1f}".format)

school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0,81.0,66.7,81.9,54.6
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1,97.0,91.3
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0,80.7,53.2
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68.3,79.3,54.3
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4,97.1,90.6
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,66.8,80.9,53.5
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.5,96.3,89.2
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.7,81.3,53.5
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.1,81.2,53.5
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6,95.9,90.5


In [243]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
all_thomas_df = all_data[all_data["school_name"] == "Thomas High School"]
thomas_all_count = len(all_thomas_df["student_name"])
thomas_10to12 = thomas_all_count - thomas_niner_count

In [244]:
# Step 6. Get all the students passing math from THS
ths_pass_math = all_data[(all_data["school_name"] == "Thomas High School") & (all_data["math_score"] > 69)]

In [245]:
# Step 7. Get all the students passing reading from THS
ths_pass_read = all_data[(all_data["school_name"] == "Thomas High School") & (all_data["reading_score"] > 69)]

In [246]:
# Step 8. Get all the students passing math and reading from THS
ths_pass_both = all_data[(all_data["reading_score"] > 69) & (all_data["math_score"] > 69)]
ths_pass_both = ths_pass_both[ths_pass_both["school_name"] == "Thomas High School"]

In [247]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School.
ths_math_perc_10to12 = format((len(ths_pass_math) / thomas_10to12 * 100), '.1f' )
#format(ths_math_perc_10to12, '.1f')

In [248]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
ths_read_perc_10to12 = format((len(ths_pass_read) / thomas_10to12 * 100), '.1f' )

In [249]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
ths_both_perc_10to12 = format((len(ths_pass_both) / float(thomas_10to12) * 100), '.1f')

In [250]:
# Step 12, 13, 14. Replace the passing percentages for THS in summary DF
school_sum_df.loc[(school_sum_df["School Type"] == "Charter") & (school_sum_df["Total Students"] == 1635), "% Pass Math"] = ths_math_perc_10to12
school_sum_df.loc[(school_sum_df["School Type"] == "Charter") & (school_sum_df["Total Students"] == 1635), "% Pass Read"] = ths_read_perc_10to12
school_sum_df.loc[(school_sum_df["School Type"] == "Charter") & (school_sum_df["Total Students"] == 1635), "% Overall Pass"] = ths_both_perc_10to12

#output
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0,81.0,66.7,81.9,54.6
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1,97.0,91.3
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0,80.7,53.2
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68.3,79.3,54.3
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4,97.1,90.6
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,66.8,80.9,53.5
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.5,96.3,89.2
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.7,81.3,53.5
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.1,81.2,53.5
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6,95.9,90.5


## delivery 2.1: best and worst performing schools per overall pass %

In [251]:
# Sort and show top five schools.

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = False)

top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1,97.0,91.3
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4,97.1,90.6
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4,83.9,93.2,97.0,90.6
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.3,84.0,93.9,96.5,90.6
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6,95.9,90.5


In [252]:
# Sort and show top five schools.

bottom_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = True)

bottom_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8,80.7,66.4,80.2,53.0
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0,80.7,53.2
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,66.8,80.9,53.5
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.7,81.3,53.5
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.1,81.2,53.5


## delivery 2.3: math and reading ave scores per grade

In [253]:
# per grade series, grouby school name, get mean

grade9 = all_data[(all_data["grade"]== "9th")]
grade10 = all_data[(all_data["grade"] == "10th" )]
grade11 = all_data[(all_data["grade"] == "11th" )]
grade12 = all_data[(all_data["grade"] == "12th" )]

grade9_math = grade9.groupby(["school_name"]).mean()["math_score"]
grade10_math = grade10.groupby(["school_name"]).mean()["math_score"]
grade11_math = grade11.groupby(["school_name"]).mean()["math_score"]
grade12_math = grade12.groupby(["school_name"]).mean()["math_score"]

grade9_read = grade9.groupby(["school_name"]).mean()["reading_score"]
grade10_read = grade10.groupby(["school_name"]).mean()["reading_score"]
grade11_read = grade11.groupby(["school_name"]).mean()["reading_score"]
grade12_read = grade12.groupby(["school_name"]).mean()["reading_score"]

#create and format math per grade per df
math_per_grade = pd.DataFrame ({
    "9th" : grade9_math,
    "10th" : grade10_math,
    "11th" : grade11_math,
    "12th" : grade12_math})

math_per_grade["9th"] = math_per_grade["9th"].map("{:.1f}".format)
math_per_grade["10th"] = math_per_grade["10th"].map("{:.1f}".format)
math_per_grade["11th"] = math_per_grade["11th"].map("{:.1f}".format)
math_per_grade["12th"] = math_per_grade["12th"].map("{:.1f}".format)

#create and format read per grade per school df
read_per_grade = pd.DataFrame ({
    "9th" : grade9_read,
    "10th" : grade10_read,
    "11th" : grade11_read,
    "12th" : grade12_read})

read_per_grade["9th"] = read_per_grade["9th"].map("{:.1f}".format)
read_per_grade["10th"] = read_per_grade["10th"].map("{:.1f}".format)
read_per_grade["11th"] = read_per_grade["11th"].map("{:.1f}".format)
read_per_grade["12th"] = read_per_grade["12th"].map("{:.1f}".format)

# correct order, remove indices if any
math_per_grade = math_per_grade[["9th", "10th", "11th", "12th"]]
read_per_grade = read_per_grade[["9th", "10th", "11th", "12th"]]
math_per_grade.index.name = None
read_per_grade.index.name = None

In [254]:
#Display the math ave scores per grade per school data frame
math_per_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [255]:
#Display the reading ave scores per grade per school data frame
read_per_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


## delivery 2.4 scores by school spending

In [256]:
#School Spending as Index
#create bins, labels, CUT data,and  add to school summary DF
spend_bins = [0,585, 630, 645, 675]
bin_label = ["<$584", "$585-629", "$630-644", "$645-675"]
ssf["Spending per Student"] = pd.cut(per_school_capita, spend_bins, labels=bin_label)

#school summary averages with spend grouping
spend_math_grade = ssf.groupby(["Spending per Student"]).mean()["Ave Math Score"]
spend_math_perc = ssf.groupby(["Spending per Student"]).mean()["% Pass Math"]
spend_read_grade = ssf.groupby(["Spending per Student"]).mean()["Ave Read Score"]
spend_read_perc = ssf.groupby(["Spending per Student"]).mean()["% Pass Read"]
spend_overall_perc = ssf.groupby(["Spending per Student"]).mean()["% Overall Pass"]

#create and format output df
spend_sum_df = pd.DataFrame({
          "Ave Math Score" : spend_math_grade,
          "Ave Read Score": spend_read_grade,
          "% Pass Math": spend_math_perc,
          "% Pass Read": spend_read_perc,
          "% Overall Pass": spend_overall_perc})

spend_sum_df["Ave Math Score"] = spend_sum_df["Ave Math Score"].map("{:.1f}".format)
spend_sum_df["Ave Read Score"] = spend_sum_df["Ave Read Score"].map("{:.1f}".format)
spend_sum_df["% Pass Math"] = spend_sum_df["% Pass Math"].map("{:.0f}".format)
spend_sum_df["% Pass Read"] = spend_sum_df["% Pass Read"].map("{:.0f}".format)
spend_sum_df["% Overall Pass"] = spend_sum_df["% Overall Pass"].map("{:.0f}".format)

#output
spend_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Spending per Student,,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,67,77,56
$645-675,77.0,81.0,66,81,54


## Scores by School Size

In [257]:
# School Size as Index
# Establish the school size bins and labels
stud_bins = [0, 1000, 2000, 5000]
bin_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
ssf["School Size"] = pd.cut(school_sum_df["Total Students"], stud_bins, labels=bin_labels)

#school summary averages with spend grouping
size_math_grade = ssf.groupby(["School Size"]).mean()["Ave Math Score"]
size_math_perc = ssf.groupby(["School Size"]).mean()["% Pass Math"]
size_read_grade = ssf.groupby(["School Size"]).mean()["Ave Read Score"]
size_read_perc = ssf.groupby(["School Size"]).mean()["% Pass Read"]
size_overall_perc = ssf.groupby(["School Size"]).mean()["% Overall Pass"]

#create and format output df
size_sum_df = pd.DataFrame({
          "Ave Math Score" : size_math_grade,
          "Ave Read Score": size_read_grade,
          "% Pass Math": size_math_perc,
          "% Pass Read": size_read_perc,
          "% Overall Pass": size_overall_perc})

size_sum_df["Ave Math Score"] = size_sum_df["Ave Math Score"].map("{:.1f}".format)
size_sum_df["Ave Read Score"] = size_sum_df["Ave Read Score"].map("{:.1f}".format)
size_sum_df["% Pass Math"] = size_sum_df["% Pass Math"].map("{:.0f}".format)
size_sum_df["% Pass Read"] = size_sum_df["% Pass Read"].map("{:.0f}".format)
size_sum_df["% Overall Pass"] = size_sum_df["% Overall Pass"].map("{:.0f}".format)

#output 
size_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,88,91,85
Large (2000-5000),77.7,81.3,70,83,58


## delivery 2.5: scores by school type

In [258]:
#School Type as Index
#No bins and labels needed, index is binary, assign value direct
type_math_grade = ssf.groupby(["School Type"]).mean()["Ave Math Score"]
type_math_perc = ssf.groupby(["School Type"]).mean()["% Pass Math"]
type_read_grade = ssf.groupby(["School Type"]).mean()["Ave Read Score"]
type_read_perc = ssf.groupby(["School Type"]).mean()["% Pass Read"]
type_overall_perc = ssf.groupby(["School Type"]).mean()["% Overall Pass"]

#create and format output df
type_sum_df = pd.DataFrame({
          "Ave Math Score" : type_math_grade,
          "Ave Read Score": type_read_grade,
          "% Pass Math": type_math_perc,
          "% Pass Read": type_read_perc,
          "% Overall Pass": type_overall_perc})

type_sum_df["Ave Math Score"] = type_sum_df["Ave Math Score"].map("{:.1f}".format)
type_sum_df["Ave Read Score"] = type_sum_df["Ave Read Score"].map("{:.1f}".format)
type_sum_df["% Pass Math"] = type_sum_df["% Pass Math"].map("{:.0f}".format)
type_sum_df["% Pass Read"] = type_sum_df["% Pass Read"].map("{:.0f}".format)
type_sum_df["% Overall Pass"] = type_sum_df["% Overall Pass"].map("{:.0f}".format)

#output
type_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Type,,,,,
Charter,83.5,83.9,90,93,87
District,77.0,81.0,67,81,54
